# Deploying Campaigns and Filters (Python SDK)

In this notebook, we'll deploy our trained recommendation models and define some filtering rules, using [Boto3, the AWS SDK for Python](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html).

> For an **alternative** approach to the same steps through the [Amazon Personalize console UI](https://console.aws.amazon.com/personalize/home) - see Notebook [04a_Deploying_Campaigns_and_Filters_(Console).ipynb](04a_Deploying_Campaigns_and_Filters_(Console).ipynb) instead.

⚠️ You'll need to already have run the previous notebooks in this series to set up your environment and train **solutions** (models) in Amazon Personalize

Before we start, we'll here:

- Import the libraries this notebook will use
- Load the variables saved from previous steps
- Connect to the relevant AWS services as we have before for IAM and S3

In [ ]:
# Python Built-Ins:
import json

# External Dependencies:
import boto3  # AWS SDK for Python

# Local Dependencies:
import util  # Small tool to print progress spinner

# Reload saved variables:
%store -r

# Connect to AWS services:
personalize = boto3.client("personalize")

## Create campaigns

A campaign is a hosted solution version: an endpoint which you can query for recommendations in real-time. Pricing is determined by throughput capacity in **transactions per second** (TPS).

When deploying a campaign you select a minimum **provisioned** TPS, which the endpoint will retain resources to serve with low-latency. However, the campaign will **auto-scale** above this if required by traffic volumes. As with many applications, autoscaling events may see some additional latency while extra infrastructure is being spun up. For more information, see the [pricing page](https://aws.amazon.com/personalize/pricing/).

While provisioning ahead may be important for serving large-scale and latency-critical demand, for initial PoCs and demos like this it will make sense for us to select the minimum provisioned TPS (1).

Let's start deploying the campaigns.

### User Personalization

Deploying the campaign for our User-Personalization solution version is fairly straightforward - a single function call.

Note that for this recipe (which includes functionality to "cold-start" recommendations for items with no or limited interactions in the historical data) we have [additional parameters](https://docs.aws.amazon.com/personalize/latest/dg/API_CreateCampaign.html) to control the trade-off between "exploring" these cold items and sticking to our ML-driven recommendations.

In [ ]:
userpersonalization_create_campaign_response = personalize.create_campaign(
    name="personalize-movielens-up",
    solutionVersionArn=userpersonalization_solution_version_arn,
    minProvisionedTPS=1,
#     campaignConfig={
#         "itemExplorationConfig": {
#             ...
#         }
#     },
)

userpersonalization_campaign_arn = userpersonalization_create_campaign_response["campaignArn"]
print(json.dumps(userpersonalization_create_campaign_response, indent=2))

> ⏰ This deployment is kicked off *in the background*, and it can take around 10 minutes to deploy a campaign.

Rather than waiting here, we'll start our other campaigns deploying first:

### SIMS

Here we'll repeat the same process for our similar items solution:

In [ ]:
sims_create_campaign_response = personalize.create_campaign(
    name="personalize-movielens-sims",
    solutionVersionArn=sims_solution_version_arn,
    minProvisionedTPS=1,
)

sims_campaign_arn = sims_create_campaign_response["campaignArn"]
print(json.dumps(sims_create_campaign_response, indent=2))

> ⏰ This deployment is kicked off *in the background*, and it can take around 10 minutes to deploy a campaign.

Rather than waiting here, we'll start our other campaigns deploying first:

### Personalized Ranking

Deploy a campaign for your personalized ranking solution version. It can take around 10 minutes to deploy a campaign. Normally, we would use a while loop to poll until the task is completed. However the task would block other cells from executing, and the goal here is to create multiple campaigns. So we will set up the while loop for all of the campaigns further down in the notebook. There, you will also find instructions for viewing the progress in the AWS console.

In [ ]:
rerank_create_campaign_response = personalize.create_campaign(
    name="personalize-movielens-rerank",
    solutionVersionArn=rerank_solution_version_arn,
    minProvisionedTPS=1,
)

rerank_campaign_arn = rerank_create_campaign_response["campaignArn"]
print(json.dumps(rerank_create_campaign_response, indent=2))

> ⏰ This deployment is kicked off *in the background*, and it can take around 10 minutes to deploy a campaign.

## Wait for Campaigns to Deploy

Of course we can view the status of these deploying campaigns through the [Amazon Personalize Console UI](https://console.aws.amazon.com/personalize/home) (See [Notebook 4a](04a_Deploying_Campaigns_and_Filters_(Console).ipynb) for more details).

...But we can also query this status through the APIs. The cell below will set up a polling loop to wait until all three campaigns have completed deployment:

In [ ]:
in_progress_campaigns = [
    userpersonalization_campaign_arn,
    sims_campaign_arn,
    rerank_campaign_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for campaign_arn in in_progress_campaigns:
        version_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = version_response["campaign"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
    
    if len(in_progress_campaigns) <= 0:
        break
    else:
        print("At least one campaign build is still in progress")
        
    time.sleep(60)

## Create Event Trackers

**Why** are we focussing on real-time model deployment with campaigns to start with, rather than batch inference? Well, recommendations can be most valuable when they're **fresh and dynamic** - updating to respond to a user's recent interactions and interests.

To get this dynamic behaviour, we'll have to **notify Personalize of new events (interactions)** as they happen - and to do that, we'll need to create an **Event Tracker**.

Creating the event tracker itself is fairly simple, and we just need to make a note of the resulting **tracking ID**:

In [ ]:
create_tracker_resp = personalize.create_event_tracker(
    name="personalize-movielens-tracker",
    datasetGroupArn=dataset_group_arn,
)

tracking_id = create_tracker_resp["trackingId"]
%store tracking_id
create_tracker_resp

We'll see the event tracker in action in the next notebook!

## Create Filters

**Filters** allow us to implement **business rules** to post-process the recommendations generated by our ML models, before they're returned from the campaign API.

As further detailed in the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/filter.html), filters can serve a range of use-cases such as:

- Filtering based on item metadata (such as genre or year)
- Filtering based on past interactions (such as excluding items the user has already watched)
- Filtering based on user metadata and combinations (such as excluding certain genres based on the customer's age or membership tier)

...Using whatever metadata fields we added to our datasets.

Filters are written in a SQL-like [Filter expression language](https://docs.aws.amazon.com/personalize/latest/dg/filter-expressions.html) - which we'll see in action here.

> ⚠️ **Note:** The Amazon Personalize console includes a **filter expression builder UI**, which may be easier to work with when first getting started than simply passing expression strings to the API as we do here!

Let's create some example filters using the fields in our sample dataset.

### 1970s Movies

For our first attempt, we'll create a filter for showing only movies from the 1970s. It's a simple function call to define our filter:

In [ ]:
create_decade_filter_resp = personalize.create_filter(
    name="1970s",
    datasetGroupArn=dataset_group_arn,
    filterExpression="INCLUDE ItemID WHERE Items.YEAR >= 1970 AND Items.YEAR < 1980",
)

### 'Watch Again' and 'Try Something New'

Next let's create two opposite filters: One to show only movies our user has watched (reviewed) before, and one to show only movies they haven't.

In [ ]:
create_watched_filter_resp = personalize.create_filter(
    name="watched",
    datasetGroupArn=dataset_group_arn,
    filterExpression='INCLUDE ItemID WHERE Interactions.event_type IN ("watch")',
)

create_unwatched_filter_resp = personalize.create_filter(
    name="unwatched",
    datasetGroupArn=dataset_group_arn,
    filterExpression='EXCLUDE ItemID WHERE Interactions.event_type IN ("watch")',
)

### Choose a Genre

Finally, we'll look at a basic example including a **parameter** you can set at request time: Building a filter for recommending from a single genre.

In [ ]:
create_genre_filter_resp = personalize.create_filter(
    name="by-genre",
    datasetGroupArn=dataset_group_arn,
    filterExpression="INCLUDE ItemID WHERE Items.GENRES IN ($GENRE)"
)

We should now have four filters set up in our dataset group:

In [ ]:
personalize.list_filters(datasetGroupArn=dataset_group_arn)

## All set!

We've now deployed our campaigns and created some filter expressions to help further refine our results.

In the next notebook we'll demonstrate **using** these models from application code to generate recommendations!

Follow along in [05_Interacting_with_Campaigns_and_Filters.ipynb](05_Interacting_with_Campaigns_and_Filters.ipynb)